# Lesson 3 Assignment - Wine Classifier

## Author - Yulia Zubova

### Instructions
Your task for this assignment:  Design a simple, low-cost sensor that can distinguish between red wine and white wine.
Your sensor must correctly distinguish between red and white wine for at least 95% of the samples in a set of 6497 test samples of red and white wine.

Your technology is capable of sensing the following wine attributes:
- Fixed acidity
- Free sulphur dioxide
- Volatile acidity  
-  Total sulphur dioxide
- Citric acid  
-  Sulphates
- Residual sugar  
-  pH
- Chlorides  
- Alcohol
- Density




## Tasks
1. Read <a href="https://library.startlearninglabs.uw.edu/DATASCI420/Datasets/WineQuality.pdf">WineQuality.pdf</a>.
2. Use the RedWhiteWine.csv or RedWhiteWine.arff that is provided.
Note: If needed, remove the quality attribute, which you will not need for this assignment.
3. Build an experiment using Naive Bayes Classifier.

Answer the following questions:
1. What is the percentage of correct classification results (using all attributes)?
2. What is the percentage of correct classification results (using a subset of the attributes)?
3. What is the AUC of your model?
4. What is the best AUC that you can achieve?
5. Which are the the minimum number of attributes? Why?


In [1]:
URL = "https://library.startlearninglabs.uw.edu/DATASCI420/Datasets/RedWhiteWine.csv"

In [2]:
# Import libraries
# import libraries
import pandas as pd
import numpy as np
import pandas_profiling
from matplotlib import pyplot
from sklearn.model_selection import KFold, cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.metrics import *
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

C:\Users\zubov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#load dataset with data about red wines 
wine_df = pd.read_csv(URL, header=0) 

In [4]:
#information about dataset
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
fixed acidity           6497 non-null float64
volatile acidity        6497 non-null float64
citric acid             6497 non-null float64
residual sugar          6497 non-null float64
chlorides               6497 non-null float64
free sulfur dioxide     6497 non-null float64
total sulfur dioxide    6497 non-null float64
density                 6497 non-null float64
pH                      6497 non-null float64
sulphates               6497 non-null float64
alcohol                 6497 non-null float64
quality                 6497 non-null int64
Class                   6497 non-null int64
dtypes: float64(11), int64(2)
memory usage: 659.9 KB


According this information, there aren't missing data in dataset.

In [5]:
#dataset statistics
wine_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Class
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,0.246114
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000,0.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000,0.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000,0.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000


In [6]:
# prints the report to the screen
pandas_profiling.ProfileReport(wine_df)

Number of variables,13
Number of observations,6497
Total Missing (%),0.0%
Total size in memory,659.9 KiB
Average record size in memory,104.0 B
Numeric,12
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,0


All variables are numerical and there aren't missing data in wine dataset.

In [7]:
# we don't need 'quality' column, let's drop it
wine_df = wine_df.drop(["quality"], axis = 1)

### All features ###

In [8]:
X = wine_df.iloc[:, 0:11]   # load all features into X DF
Y = wine_df.iloc[:, 11]     # Load target into Y DF

In [9]:
# Add with viewing the data
pd.set_option('display.width', 100) 
pd.set_option('precision', 2)

In [10]:
#boxplots for each variable
wine_df.plot(kind='box', subplots=True, layout=(5,3), sharex=False, sharey=False, figsize=(12,15))
pyplot.show()

In [11]:
kfold = KFold(n_splits=15, random_state=7)  # 10 fold cross validation ; 
                                            # 7 random state is to assure consistent results

In [12]:
#Getting cross validation score for Naive Bayes Classifier using all features
nbc_results = cross_val_score(GaussianNB(), X, Y, cv=kfold)
print("Accuracy: %.3f%% (std:%.3f)" % (nbc_results.mean()*100, nbc_results.std()))

Accuracy: 96.876% (std:0.011)


#### The same actions but without cross validation ####

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2) #split data to train and test

In [14]:
clf = GaussianNB() # with default parameters
#Train classification model
nbc_model = clf.fit(x_train, y_train)
#Apply model and get predictions
y_pred = nbc_model.predict(x_test)

In [15]:
misclassified_points = (y_test != y_pred).sum()
print("Number of mislabeled points out of a total %d points : %d"\
      % (y_test.shape[0], misclassified_points))
print("Accuracy = %.2f"%(round((y_test.shape[0] - float(misclassified_points))/y_test.shape[0]*100,2)))

Number of mislabeled points out of a total 1300 points : 33
Accuracy = 97.46


In [16]:
#Confusion matrix

print(confusion_matrix(y_test, y_pred))

CM_log = confusion_matrix(y_test, y_pred)

[[964  25]
 [  8 303]]


In [17]:
#Getting performance metrics
report = classification_report(y_test, y_pred)
print(report)

             precision    recall  f1-score   support

          0       0.99      0.97      0.98       989
          1       0.92      0.97      0.95       311

avg / total       0.98      0.97      0.97      1300



In [25]:
#AUC score for this model
print(roc_auc_score(y_test, y_pred))

0.974499234343


Naive Bayes Classifier shows great results if we use all features (Accuracy, Precision, Recall and AUC are very close to 1). 

### Perform some feature selection ###

On correlation matrix (above) we can notice that several variables are highly correlated with each other. I'm going to delete "free sulfur dioxide", "density" from dataset and check performance.

In [26]:
#delete columns from X matrix (features) 
X2 = X.drop(["free sulfur dioxide", "density"], axis = 1)


In [27]:
#Getting cross validation score for Naive Bayes Classifier
nbc_results2 = cross_val_score(GaussianNB(), X2, Y, cv=kfold)
print("Accuracy: %.3f%% (std:%.3f)" % (nbc_results2.mean()*100, nbc_results2.std()))

Accuracy: 96.198% (std:0.013)


In [28]:
#Split X and Y matrices to train (80%) and test (20%) sets
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y, test_size = 0.2)

In [29]:
# Classifier- Naive Bayes
clf = GaussianNB() # with default parameters
#Train classification model
nbc_model2 = clf.fit(x_train2, y_train2)
#Apply model and get predictions
y_pred2 = nbc_model2.predict(x_test2)

In [30]:
misclassified_points = (y_test2 != y_pred2).sum()
print("Number of mislabeled points out of a total %d points : %d"\
      % (y_test2.shape[0], misclassified_points))
print("Accuracy = %.2f"%(round((y_test2.shape[0] - float(misclassified_points))/y_test2.shape[0]*100,2)))

Number of mislabeled points out of a total 1300 points : 41
Accuracy = 96.85


In [31]:
#Confusion matrix
print(confusion_matrix(y_test2, y_pred2))

CM_log = confusion_matrix(y_test2, y_pred2)

[[928  32]
 [  9 331]]


In [32]:
#Getting performance metrics
report = classification_report(y_test2, y_pred2)
print(report)

             precision    recall  f1-score   support

          0       0.99      0.97      0.98       960
          1       0.91      0.97      0.94       340

avg / total       0.97      0.97      0.97      1300



In [33]:
print(roc_auc_score(y_test2, y_pred2))

0.970098039216


Model showed good performance too after deleting two variables, but a little lower (less then 1%) in comparison with model where all features were used.

### K-best feature selection ###

In [60]:
# feature extraction K-best selection method
test = SelectKBest(score_func=chi2, k=5)
fit = test.fit(X, Y)
# summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)  #get scores (importance) for each feature
features = fit.transform(X)


[  3.585e+02   2.211e+02   1.512e+01   3.287e+03   3.740e+01   1.491e+04
   8.795e+04   8.961e-03   5.652e+00   6.427e+01   9.574e-01]


According the results, the order of importance of features (starting with the most important) is:
- total sulfur dioxide
- free sulfur dioxide
- residual sugar
- fixed acidity
- volatile acidity
- sulphates
- chlorides
- citric acid
- pH
- alcohol
- density.

I've made several experiments, and target accuracy 95% was achieved using at least 7 variables ('residual sugar',  'free sulfur dioxide', 'total sulfur dioxide', 'fixed acidity', 'volatile acidity', 'sulphates', 'chlorides')

In [76]:
X.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'],
      dtype='object')

In [61]:
X3 = X[['residual sugar',  'free sulfur dioxide', 'total sulfur dioxide', 'fixed acidity', 'volatile acidity', 'sulphates', 
        'chlorides']]

In [59]:
#Getting cross validation score for Naive Bayes Classifier
nbc_results3 = cross_val_score(GaussianNB(), X3, Y, cv=kfold)
print("Accuracy: %.3f%% (std:%.3f)" % (nbc_results3.mean()*100, nbc_results3.std()))

Accuracy: 95.690% (std:0.015)


In [64]:
#Split X and Y matrices to train (80%) and test (20%) sets
x_train3, x_test3, y_train3, y_test3 = train_test_split(X3, Y, test_size = 0.2)

In [65]:
# Classifier- Naive Bayes
clf = GaussianNB() # with default parameters
#Train classification model
nbc_model3 = clf.fit(x_train3, y_train3)
#Apply model and get predictions
y_pred3 = nbc_model3.predict(x_test3)

In [66]:
misclassified_points = (y_test3 != y_pred3).sum()
print("Number of mislabeled points out of a total %d points : %d"\
      % (y_test3.shape[0], misclassified_points))
print("Accuracy = %.2f"%(round((y_test3.shape[0] - float(misclassified_points))/y_test3.shape[0]*100,2)))

Number of mislabeled points out of a total 1300 points : 60
Accuracy = 95.38


In [67]:
#Confusion matrix
print(confusion_matrix(y_test3, y_pred3))

CM_log = confusion_matrix(y_test3, y_pred3)

[[935  43]
 [ 17 305]]


In [70]:
#Getting performance metrics
report = classification_report(y_test3, y_pred3)
print(report)

             precision    recall  f1-score   support

          0       0.98      0.96      0.97       978
          1       0.88      0.95      0.91       322

avg / total       0.96      0.95      0.95      1300



In [71]:
print(roc_auc_score(y_test3, y_pred3))

0.95161884439


## SUMMARY ##

1. Accuracy (percentage of correct classification results):
    - using all features Accuracy = 96.876%
    - after some "intuitive" feature selection  Accuracy = 96.198%
    - after feature selection based on k-best selection Accuracy = 95.690%


2. AUC - the area under the the ROC curve. ROC AUC varies between 0 and 1 — with an uninformative classifier yielding 0.5.
So the best possible value ROC AUC = 1.
    - using all features ROC AUC = 0.974
    - after some "intuitive" feature selection  ROC AUC = 0.970
    - after feature selection based on k-best selection ROC AUC = 0.952.
    
3. Minimum amount of features that we need to use for getting at least 95% correct predictions is 7 ('residual sugar',  'free sulfur dioxide', 'total sulfur dioxide', 'fixed acidity', 'volatile acidity', 'sulphates', 'chlorides'). 
This subset was gotten using k-best selection method and following experiments.

In general, Naive Bayes Classificator performed very well using k-best method, and show the best results 
if we use all features.